# 🚀 Analyse Rhétorique Collaborative par Agents IA - Exécuteur Principal

**Objectif:** Ce notebook orchestre et exécute une analyse rhétorique multi-agents sur un texte donné. Il sert de point d'entrée principal pour lancer le processus.

**Structure Modulaire:**
1.  `Argument_Analysis_UI_configuration.ipynb` : Gère l'interface utilisateur pour sélectionner/préparer le texte à analyser (incluant sources prédéfinies, URL, fichier, texte direct, et extraction) et charge/sauvegarde la configuration des sources.
2.  `Argument_Analysis_Agentic-0-init.ipynb`: Configuration initiale (dépendances, LLM, JVM), définition de l'état partagé (`RhetoricalAnalysisState`) et du gestionnaire d'état (`StateManagerPlugin`).
3.  `Argument_Analysis_Agentic-1-informal_agent.ipynb`: Définition de l'`InformalAnalysisAgent`.
4.  `Argument_Analysis_Agentic-2-pl_agent.ipynb`: Définition du `PropositionalLogicAgent`.
5.  `Argument_Analysis_Agentic-3-orchestration.ipynb`: Définition des stratégies d'orchestration et de la fonction principale `run_analysis_conversation`.

**Prérequis:**
* Un fichier `.env` à la racine contenant les clés API, configurations LLM, et la clé de chiffrement `TEXT_CONFIG_KEY`.
* Un environnement Java Development Kit (JDK >= 11) correctement installé et configuré (`JAVA_HOME`).
* Les dépendances Python installées (`ipywidgets`, `requests`, `jupyter-ui-poll`, `python-dotenv`, `semantic-kernel`, `pandas`, `jpype1`, `cryptography`).
* Les JARs Tweety placés dans le dossier `libs/`.
* Le fichier `extract_sources.json.gz.enc` (s'il existe déjà) contenant les définitions des sources.

## 1. Chargement de l'Environnement

Chargement des variables depuis le fichier `.env` (clés API, clé de chiffrement, etc.).

In [ ]:
# Charger les variables d'environnement
from dotenv import load_dotenv, find_dotenv
loaded_env = load_dotenv(find_dotenv(), override=True)
print(f".env chargé: {loaded_env}") # Affiche True si le .env a été trouvé et chargé

## 2. Chargement de l'Interface Utilisateur

Exécution du notebook `UI_Configuration.ipynb` pour définir la fonction `configure_analysis_task()`. C'est ce notebook qui contient désormais toute la logique de l'interface graphique, du cache fichier et de la gestion de la configuration chiffrée.

In [ ]:
# Mode batch contrôlé par variable d'environnement
# Si BATCH_MODE=true dans .env, on skip l'UI interactive (widgets bloquants)
import os
BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() in ("true", "1", "yes")

if BATCH_MODE:
    print("Mode BATCH detecte (BATCH_MODE=true dans .env)")
    print("   -> Skip du chargement UI_configuration.ipynb (widgets non compatibles)")
    print("   -> Le texte sera fourni directement dans la cellule suivante")
else:
    # Exécuter le notebook UI pour définir la fonction configure_analysis_task
    # Assurez-vous que le fichier UI_Configuration.ipynb est dans le même répertoire
    print("Exécution de Argument_Analysis_UI_configuration.ipynb...")
    %run ./Argument_Analysis_UI_configuration.ipynb
    print("Exécution de Argument_Analysis_UI_configuration.ipynb terminée.")

    # Vérification que la fonction est bien définie après l'exécution
    if 'configure_analysis_task' not in locals():
        print("ERREUR CRITIQUE : La fonction configure_analysis_task n'a pas été définie par UI_Configuration.ipynb !")
    else:
        print("Fonction configure_analysis_task trouvée.")

## 3. Configuration de la Tâche et Récupération du Texte

Appel de la fonction `configure_analysis_task()` définie dans le notebook UI. Cela affichera l'interface utilisateur. Sélectionnez votre source, préparez le texte, puis cliquez sur **"Lancer l'Analyse"**. Le texte préparé sera retourné et stocké pour l'étape suivante. La cellule attendra la fin de votre interaction avec l'UI.

In [ ]:
# Configuration du texte pour l'analyse
# En mode BATCH : utilise BATCH_TEXT (variable d'env) ou le texte d'exemple
# En mode interactif : appelle configure_analysis_task() (si disponible)

texte_pour_analyse = None

# Texte d'exemple enrichi pour le mode batch - contient des arguments complexes et des sophismes
TEXTE_EXEMPLE_BATCH = """Le debat sur la transition energetique en France revele des enjeux majeurs pour notre avenir.

ARGUMENTS EN FAVEUR DES ENERGIES RENOUVELABLES:
Les energies renouvelables sont essentielles pour lutter contre le changement climatique. La France s'est engagee a reduire ses emissions de CO2 de 40% d'ici 2030, ce qui necessite un investissement massif dans le solaire et l'eolien. D'ailleurs, l'Allemagne a deja prouve que c'etait possible en produisant 46% de son electricite a partir de sources renouvelables en 2023.

De plus, les energies renouvelables creent des emplois locaux non delocalisables. Selon l'ADEME, le secteur emploie deja plus de 150 000 personnes en France et pourrait en creer 500 000 supplementaires d'ici 2050. Quiconque s'oppose a cette transition est donc contre la creation d'emplois.

ARGUMENTS CONTRE LE DEVELOPPEMENT MASSIF DES RENOUVELABLES:
Cependant, les ecologistes veulent nous ramener a l'age de pierre en supprimant le nucleaire, qui fournit pourtant 70% de notre electricite. Sans le nucleaire, nous serions obliges de choisir entre des coupures d'electricite massives ou une dependance totale au gaz russe.

Le cout de la transition est exorbitant. Tout le monde sait que les eoliennes sont inefficaces car elles ne tournent que 25% du temps. Mon voisin Jean-Pierre, qui est ingenieur a EDF, m'a dit que les renouvelables ne pourront jamais remplacer le nucleaire. C'est un expert, donc il a forcemment raison.

De plus, les panneaux solaires sont fabriques en Chine avec du charbon, ce qui annule completement leur benefice ecologique. Donc investir dans le solaire revient a financer la pollution chinoise.

CONCLUSION:
Il faut arreter de debattre et agir. Soit on accepte une transition energetique complete vers 100% de renouvelables d'ici 2040, soit on condamne nos enfants a vivre sur une planete inhabitable. Des milliers de scientifiques ont signe une petition pour le climat, ce qui prouve definitivement que nous avons raison de promouvoir les energies vertes. Les climatosceptiques sont finances par les lobbies petroliers et ne meritent pas qu'on ecoute leurs arguments.

En conclusion, bien que certains arguments des deux cotes aient du merite, la question necessite une analyse nuancee des compromis entre independance energetique, impact environnemental, cout economique et faisabilite technique."""

if BATCH_MODE:
    # === MODE BATCH ===
    print("Mode BATCH - Configuration automatique du texte")
    
    # Priorite 1: Variable d'environnement BATCH_TEXT
    batch_text_env = os.getenv("BATCH_TEXT", "")
    if batch_text_env:
        texte_pour_analyse = batch_text_env
        print(f"Texte charge depuis BATCH_TEXT ({len(texte_pour_analyse)} caracteres)")
    else:
        # Priorite 2: Texte d'exemple par defaut
        texte_pour_analyse = TEXTE_EXEMPLE_BATCH
        print(f"Texte d'exemple enrichi utilise ({len(texte_pour_analyse)} caracteres)")
    
    print(f"\nExtrait du texte:\n{texte_pour_analyse[:200]}...")

else:
    # === MODE INTERACTIF ===
    print("Mode INTERACTIF - Lancement de l'interface de configuration")
    
    if 'configure_analysis_task' in locals():
        try:
            texte_pour_analyse = configure_analysis_task()
            print(f"Texte recupere via l'interface ({len(texte_pour_analyse) if texte_pour_analyse else 0} caracteres)")
        except Exception as e_ui:
            print(f"Erreur lors de la configuration UI : {e_ui}")
            import traceback
            traceback.print_exc()
    else:
        print("Fonction configure_analysis_task non disponible - verifiez le chargement de UI_configuration.ipynb")

# Verification finale
if not texte_pour_analyse:
    print("\nAucun texte prepare. L'analyse ne peut pas continuer.")
else:
    print(f"\nTexte pret pour l'analyse (longueur: {len(texte_pour_analyse)}). Passage au chargement des agents.")

## 4. Chargement des Définitions des Agents et de l'Orchestration

Maintenant que le texte est prêt (si l'étape précédente a réussi), nous chargeons les définitions des agents, des plugins, des stratégies et de la fonction d'orchestration `run_analysis_conversation` en exécutant les notebooks enfants dédiés.

**Rappel:** Le notebook `Argument_Analysis_Agentic-0-init.ipynb` **ne doit plus définir** la variable `raw_text_input` et le notebook `Argument_Analysis_Agentic-3-orchestration.ipynb` **doit définir** `run_analysis_conversation(texte_a_analyser)` acceptant un argument.

In [ ]:
# Exécuter les notebooks enfants pour charger les définitions
# Seulement si un texte a été préparé avec succès
if 'texte_pour_analyse' in locals() and texte_pour_analyse:
    print("\nChargement des définitions des agents et de l'orchestration...")
    try:
        %run ./Argument_Analysis_Agentic-0-init.ipynb
        %run ./Argument_Analysis_Agentic-1-informal_agent.ipynb
        %run ./Argument_Analysis_Agentic-2-pl_agent.ipynb
        %run ./Argument_Analysis_Agentic-3-orchestration.ipynb  # Définit run_analysis_conversation(texte_a_analyser)
        print("✅ Définitions chargées.")
        # Vérifier que la fonction d'orchestration est chargée
        if 'run_analysis_conversation' not in locals():
             print("❌ ERREUR CRITIQUE: La fonction run_analysis_conversation n'a pas été définie après l'exécution des notebooks agents!")
             # raise NameError("run_analysis_conversation non définie")
    except Exception as e_run:
        print(f"\n❌ Une erreur est survenue lors de l'exécution des notebooks enfants : {e_run}")
        import traceback
        traceback.print_exc()
        # Empêcher la suite si le chargement échoue
        texte_pour_analyse = None
else:
    print("\nSkipping agent definition loading because no text was prepared.")

## 5. Exécution de l'Analyse Collaborative

Si toutes les étapes précédentes se sont bien déroulées et que nous avons un texte à analyser, cette cellule lance l'analyse collaborative.

*Note :* `nest_asyncio` est appliqué pour la compatibilité avec l'environnement asynchrone de Jupyter.

In [ ]:
import nest_asyncio
import asyncio

# Lancer seulement si on a obtenu un texte valide ET que les définitions sont chargées
if 'texte_pour_analyse' in locals() and texte_pour_analyse and 'run_analysis_conversation' in locals():
    print("\n🚀 Lancement de l'exécution asynchrone de l'analyse...")
    nest_asyncio.apply()
    try:
        # Passer le texte préparé
        local_state = await run_analysis_conversation(texte_pour_analyse)
        print("\n🏁 Analyse terminée.")
    except Exception as e_analysis:
        print(f"\n❌ Une erreur est survenue pendant l'exécution de l'analyse : {e_analysis}")
        import traceback
        traceback.print_exc()

elif 'texte_pour_analyse' not in locals() or not texte_pour_analyse:
    print("\n Analyse non lancée : aucun texte n'a été préparé ou une erreur est survenue avant.")
else: # Implique que run_analysis_conversation n'a pas été chargée
     print("\n Analyse non lancée : la fonction d'orchestration n'a pas pu être chargée.")

## 5bis. Rapport de Validation de l'Analyse

Cette cellule genere un rapport structure validant la completude de l'analyse rhetorique.
Elle verifie les criteres suivants:
- Arguments identifies
- Sophismes analyses
- Belief Sets PL crees
- Requetes logiques executees
- Conclusion generee

Le rapport est exporte en JSON pour utilisation ulterieure.

In [ ]:
# === CELLULE DE VALIDATION FINALE ===
# Genere un rapport JSON structure avec cross-validation

import json
from datetime import datetime
from typing import Dict, Any, List, Optional

def generate_validated_analysis_report(state) -> Dict[str, Any]:
    """Genere rapport JSON structure avec cross-validation."""
    
    report = {
        "metadata": {
            "timestamp": datetime.now().isoformat(),
            "version": "2.0-validated",
            "text_length": len(state.raw_text) if state.raw_text else 0,
            "text_snippet": (state.raw_text[:150] + "...") if state.raw_text and len(state.raw_text) > 150 else (state.raw_text or "")
        },
        "informal_analysis": {
            "arguments": [],
            "fallacies": [],
            "taxonomy_families_used": set()
        },
        "formal_analysis": {
            "belief_sets": [],
            "query_results": [],
            "consistency_checked": False
        },
        "cross_validation": {
            "validation_status": "INCOMPLETE",
            "confidence_score": 0.0,
            "checks_passed": [],
            "issues": []
        },
        "conclusion": {
            "summary": state.final_conclusion if hasattr(state, 'final_conclusion') else None,
            "is_complete": hasattr(state, 'final_conclusion') and state.final_conclusion is not None
        }
    }
    
    # Populate arguments
    if hasattr(state, 'identified_arguments'):
        for arg_id, arg_desc in state.identified_arguments.items():
            has_fallacy = False
            if hasattr(state, 'identified_fallacies'):
                has_fallacy = any(
                    f.get('target_argument_id') == arg_id
                    for f in state.identified_fallacies.values()
                )
            report["informal_analysis"]["arguments"].append({
                "id": arg_id, "description": str(arg_desc)[:200], "has_fallacy": has_fallacy
            })
    
    # Populate fallacies
    if hasattr(state, 'identified_fallacies'):
        for f_id, f_data in state.identified_fallacies.items():
            fallacy_type = f_data.get('type', 'Unknown') if isinstance(f_data, dict) else str(f_data)
            report["informal_analysis"]["fallacies"].append({
                "id": f_id,
                "type": fallacy_type,
                "justification": f_data.get('justification', '') if isinstance(f_data, dict) else '',
                "target_id": f_data.get('target_argument_id') if isinstance(f_data, dict) else None,
                "severity": "HIGH" if any(kw in str(fallacy_type).lower() for kw in ['manipulation', 'tromperie']) else "MEDIUM"
            })
            # Track taxonomy families
            if isinstance(fallacy_type, str) and '/' in fallacy_type:
                report["informal_analysis"]["taxonomy_families_used"].add(fallacy_type.split('/')[0])
    
    report["informal_analysis"]["taxonomy_families_used"] = list(report["informal_analysis"]["taxonomy_families_used"])
    
    # Populate belief sets
    if hasattr(state, 'belief_sets'):
        for bs_id, bs_data in state.belief_sets.items():
            content = bs_data.get('content', '') if isinstance(bs_data, dict) else str(bs_data)
            report["formal_analysis"]["belief_sets"].append({
                "id": bs_id,
                "logic_type": bs_data.get('logic_type', 'PL') if isinstance(bs_data, dict) else 'PL',
                "formula_count": content.count('\n') + 1 if content else 0,
                "is_consistent": "NOT_CHECKED"
            })
    
    # Populate query results
    if hasattr(state, 'query_log'):
        for qlog in state.query_log:
            raw_result = qlog.get('raw_result', '') if isinstance(qlog, dict) else ''
            status = "UNKNOWN"
            if "ACCEPTED" in str(raw_result): status = "ACCEPTED"
            elif "REJECTED" in str(raw_result): status = "REJECTED"
            elif "FUNC_ERROR" in str(raw_result): status = "ERROR"
            
            report["formal_analysis"]["query_results"].append({
                "log_id": qlog.get('log_id', '') if isinstance(qlog, dict) else '',
                "belief_set_id": qlog.get('belief_set_id', '') if isinstance(qlog, dict) else '',
                "query": qlog.get('query', '') if isinstance(qlog, dict) else '',
                "status": status
            })
    
    # === CROSS-VALIDATION LOGIC ===
    checks = []
    issues = []
    
    # Check 1: Arguments identified
    if len(report["informal_analysis"]["arguments"]) > 0:
        checks.append("ARGUMENTS_IDENTIFIED")
    else:
        issues.append("Aucun argument identifie")
    
    # Check 2: Fallacy analysis attempted
    if len(report["informal_analysis"]["fallacies"]) > 0:
        checks.append("FALLACIES_ANALYZED")
    elif hasattr(state, 'answers') and any("sophisme" in str(v).lower() for v in state.answers.values()):
        checks.append("FALLACY_ANALYSIS_ATTEMPTED")
    else:
        issues.append("Analyse sophismes non effectuee")
    
    # Check 3: Formal logic translation
    if len(report["formal_analysis"]["belief_sets"]) > 0:
        checks.append("BELIEF_SET_CREATED")
    else:
        issues.append("Aucun Belief Set PL cree")
    
    # Check 4: Queries executed
    if len(report["formal_analysis"]["query_results"]) > 0:
        checks.append("QUERIES_EXECUTED")
        accepted = sum(1 for q in report["formal_analysis"]["query_results"] if q["status"] == "ACCEPTED")
        rejected = sum(1 for q in report["formal_analysis"]["query_results"] if q["status"] == "REJECTED")
        if accepted > 0 or rejected > 0:
            checks.append("QUERIES_MEANINGFUL")
    else:
        issues.append("Aucune requete PL executee")
    
    # Check 5: Conclusion generated
    if report["conclusion"]["is_complete"]:
        checks.append("CONCLUSION_GENERATED")
    else:
        issues.append("Conclusion finale non generee")
    
    # Calculate confidence score
    max_checks = 6  # ARGUMENTS, FALLACIES, BELIEF_SET, QUERIES, QUERIES_MEANINGFUL, CONCLUSION
    confidence = len(checks) / max_checks
    report["cross_validation"]["confidence_score"] = round(confidence, 2)
    report["cross_validation"]["checks_passed"] = checks
    report["cross_validation"]["issues"] = issues
    
    # Determine validation status
    if confidence >= 0.8:
        report["cross_validation"]["validation_status"] = "COMPLETE_VALIDATED"
    elif confidence >= 0.5:
        report["cross_validation"]["validation_status"] = "PARTIAL_VALIDATED"
    elif confidence >= 0.3:
        report["cross_validation"]["validation_status"] = "MINIMAL"
    else:
        report["cross_validation"]["validation_status"] = "INCOMPLETE"
    
    return report


def display_validation_summary(report: Dict[str, Any]) -> str:
    """Affiche resume lisible du rapport de validation."""
    print("\n" + "="*70)
    print("          RAPPORT D'ANALYSE RHETORIQUE VALIDEE")
    print("="*70)
    
    cv = report["cross_validation"]
    status_symbols = {
        "COMPLETE_VALIDATED": "[OK]",
        "PARTIAL_VALIDATED": "[PARTIEL]",
        "MINIMAL": "[MINIMAL]",
        "INCOMPLETE": "[INCOMPLET]"
    }
    
    print(f"\n  STATUT: {status_symbols.get(cv['validation_status'], '?')} {cv['validation_status']}")
    print(f"  CONFIANCE: {cv['confidence_score']*100:.0f}%")
    
    print(f"\n  [ANALYSE INFORMELLE]")
    print(f"    Arguments: {len(report['informal_analysis']['arguments'])}")
    print(f"    Sophismes: {len(report['informal_analysis']['fallacies'])}")
    
    print(f"\n  [ANALYSE FORMELLE]")
    print(f"    Belief Sets: {len(report['formal_analysis']['belief_sets'])}")
    print(f"    Requetes: {len(report['formal_analysis']['query_results'])}")
    
    print(f"\n  [VALIDATIONS PASSEES]")
    for check in cv['checks_passed']:
        print(f"    [+] {check}")
    
    if cv['issues']:
        print(f"\n  [PROBLEMES DETECTES]")
        for issue in cv['issues']:
            print(f"    [-] {issue}")
    
    print(f"\n  [CONCLUSION]")
    if report['conclusion']['is_complete']:
        conclusion_preview = str(report['conclusion']['summary'])[:200]
        print(f"    {conclusion_preview}...")
    else:
        print("    (Non generee)")
    
    print("\n" + "="*70)
    
    return cv['validation_status']


# === EXECUTION DE LA VALIDATION ===
print("\n--- Generation du Rapport d'Analyse Validee ---")

# Chercher l'etat local_state defini par l'orchestration
if 'local_state' in dir() and local_state is not None:
    # Generate report
    final_report = generate_validated_analysis_report(local_state)
    
    # Display summary
    validation_status = display_validation_summary(final_report)
    
    # Export JSON
    output_path = "output/analysis_report.json"
    try:
        import os
        os.makedirs("output", exist_ok=True)
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(final_report, f, indent=2, ensure_ascii=False, default=str)
        print(f"\nRapport JSON exporte vers: {output_path}")
    except Exception as e:
        print(f"Erreur export JSON: {e}")
    
    # Afficher JSON complet
    print("\n--- RAPPORT JSON COMPLET ---")
    print(json.dumps(final_report, indent=2, ensure_ascii=False, default=str))
    
    # Final verdict
    print("\n--- VERDICT FINAL ---")
    if validation_status == "COMPLETE_VALIDATED":
        print("[SUCCESS] ANALYSE RHETORIQUE VALIDEE COMPLETE")
        print("  Tous les criteres de validation sont satisfaits.")
    elif validation_status == "PARTIAL_VALIDATED":
        print("[PARTIEL] Analyse partiellement validee")
        print("  Certaines etapes manquent pour une validation complete.")
    else:
        print(f"[{validation_status}] Analyse incomplete")
        print("  Verifiez les problemes detectes ci-dessus.")

else:
    print("[INFO] Etat d'analyse (local_state) non disponible.")
    print("  Executez d'abord les cellules precedentes pour lancer l'analyse.")
    print("  Ou l'analyse n'a pas ete executee (mode batch sans erreur?)")

## 6. Résultats et Conclusion

Vérifiez les logs et l'état final JSON affichés par l'exécution précédente pour voir le résultat de l'analyse collaborative.

## 7. 🏁 Pistes d'Amélioration Futures

*(Repris de `3-orchestration-...ipynb`)*

**Prochaines étapes possibles :**
* **Activer & Finaliser PL:** Implémenter réellement les appels JPype/Tweety dans `PropositionalLogicPlugin._internal_execute_query` et tester de bout en bout l'exécution des requêtes logiques (parsing, query, interprétation).
* **Affiner Analyse Sophismes:** Améliorer les instructions de `InformalAnalysisAgent` pour une exploration plus fine de la taxonomie (gestion de la profondeur, choix des branches) ou l'attribution de sophismes spécifiques basée sur les détails récupérés (`get_fallacy_details`).
* **Externaliser Prompts & Config:** Déplacer les prompts et configurations (ex: noms agents, constantes) hors du code Python vers des fichiers dédiés (YAML, JSON, .env) pour une meilleure maintenabilité. Utiliser `kernel.import_plugin_from_directory`.
* **Gestion Erreurs Agents:** Renforcer la capacité des agents à gérer les erreurs retournées par les outils (`FUNC_ERROR:`) et à adapter leur plan (ex: demander une clarification, réessayer, passer à autre chose).
* **Nouveaux Agents/Capacités:** Implémenter des agents pour d'autres logiques (FOL, Modale), d'autres tâches (résumé, extraction d'entités) ou d'autres outils (recherche web, base de données).
* **État RDF/KG:** Explorer le passage à une structure d'état plus riche et sémantiquement structurée en utilisant RDF/KG (avec `rdflib` ou une base de graphe) pour représenter les arguments, relations, et métadonnées de manière plus formelle.
* **Interface Utilisateur:** Créer une interface (ex: avec Gradio, Streamlit) pour faciliter l'interaction et la visualisation de l'analyse.